<h3>Step 1: Imputation tried before using mice. Manually tried imputing the values using by runnning linear regression and taking a mean of the values. But we were instructed not use this method.</h3>
    
Logic:<br>
    <l>Check for each year.<br>
    <l>If missing check 3 years before and 3 years after.<br>
    <l>Perform linear regresssion and predict value for each avaialble years.<br>
    <l>Take a mean of those values</l>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import math


In [30]:
def impute(source_year,target_year,raw_data,indicator,year):
    
    #Pulling out data specific to the indicator and the year
    #print("Predicting indicator{0} data for {1} with {2} data:".format(indicator,target_year,source_year))
    raw_data=raw_data[raw_data["Indicator Code"]==indicator]
    count_nan_columns = len(raw_data) - raw_data.count()
    count_nan_columns=count_nan_columns[str(year)]
    print(indicator,": ",count_nan_columns)
    
    reduced_data=raw_data[[str(source_year),str(target_year)]]
    
    #Removing records where source data is null since they can't be used for traning
    filtered_data = reduced_data[reduced_data[str(source_year)].notnull()] 
    
    #Building a dataframe with null target values to fill
    data_to_filled= filtered_data[filtered_data[str(target_year)].isnull()]
    
    #creating a training data/test dataset filtering out null values in target:
    final_dataset = filtered_data[filtered_data[str(target_year)].notnull()] 
    
    #if there is no data with target year null and valid data in source year skip it
    if(len(final_dataset)==0):
        #print("No data to predict")
        print("Predicted:",0)
        return[]
    #Correlation gives an idea of how target year is realted to the source year
    corr=final_dataset[str(source_year)].corr(final_dataset[str(target_year)]) 
    #print("Cor:",corr)##0.9998088986272333 for 2009 and 2010
    
    #function which defines the linear model
    final=linear(final_dataset,source_year,target_year,data_to_filled)
    print("Predicted:", len(final))
    return(final)

In [31]:
def linear(final_dataset,source_year,target_year,data_to_filled):
    #split it into train and test dataset to test model efficiency
    X_train, X_test, y_train, y_test = train_test_split(final_dataset[[str(source_year)]], final_dataset[[str(target_year)]], test_size=0.0)
    #print ("Total data:",final_dataset.shape)
    #print ("Train data:",X_train.shape, y_train.shape)
    #print ("Test data: ",X_test.shape, y_test.shape)
   
    #Training a linear model
    lm = linear_model.LinearRegression()
    if(len(X_train)==0):
        #print("0 values predicted")
        return([])
    model = lm.fit(X_train, y_train)
    
    #Testing model
    #predictions = lm.predict(X_test)
    #print("Score:",model.score(X_test, y_test))
    
    #Using the model to fit missing values in target data
    final_data=data_to_filled[[str(source_year)]]
    if(len(final_data)==0):
        #print("0 values predicted")
        return([])
    final=lm.predict(final_data)
    
    #Skim through th rows to fill missing data using available years
    data_to_filled[str(target_year)+"_"+str(source_year)] = data_to_filled.apply(lambda row : lm.predict([[row[str(source_year)]]]), axis = 1)
    data_to_filled[str(target_year)+"_"+str(source_year)] = data_to_filled.apply(lambda row : row[str(target_year)+"_"+str(source_year)][0][0], axis = 1)
    #print(len(data_to_filled), "values predicted")
    
    return(data_to_filled)

In [32]:
def prep_file(file,year,number):
    #read csv file
    raw_data=pd.read_csv("./"+file,skiprows=4)
    list_of_indicators=raw_data["Indicator Code"].unique()
    imputation_data_frame=pd.DataFrame()  
    #create the imputation dataframe
    for years in range(year-number,year+number+1):
        if(years==year):
            continue
        for i,indi in enumerate(list_of_indicators):
            data=impute(years,year,raw_data,indi,year)
            if(len(data)==0):
                continue
            imputation_data_frame=imputation_data_frame.append(data)
    #Adds(all other records for the same index under in the year has to be 0) all data grouped by index creating just one record per index. 
    imputation_data_frame=imputation_data_frame.groupby(imputation_data_frame.index).sum()
    
    #calculates the mean of data vailable
    imputation_data_frame[str(year)] = imputation_data_frame.apply(lambda row : funcmean(row), axis = 1)
    
    #replace nan in original dataset with imputed values
    for i,row in raw_data.iterrows():
        if(math.isnan(row[str(year)])):
            if i in imputation_data_frame.index:
                imputed=imputation_data_frame.loc[i][str(year)]
                if(math.isnan(imputed)):
                    continue
                else: 
                    raw_data.at[i, str(year)] = imputed
    return(raw_data)

In [33]:
#calaculate mean of the years
def funcmean(row):
    mean=0
    count=0
    array=[]
    for i in row.index:
        if "_" not in i:
            row=row.drop(i)
        elif(row[i]==0):
            row=row.drop(i)
    return(row.mean())

In [34]:
def finalRunImputation(file,year,number):
    test=prep_file(file,year,number)
    test=test[[str(year),"Country Name","Country Code","Indicator Name","Indicator Code"]]
    test = test[np.isfinite(test['2010'])]
    test.to_csv("new"+file, encoding='utf-8', index=False)

In [ ]:
os.getcwd()
#get all files in the current folder and run a function to create a dict with file names linked to the data sets
for file in os.listdir():
    if ".csv" in file:
        finalRunImputation(file,2010,3)
        

SE.PRM.UNER.ZS :  94
Predicted: 22
SE.PRM.UNER.MA.ZS :  110
Predicted: 28
SE.PRM.UNER.MA :  110


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Predicted: 28
SE.PRM.UNER.FE.ZS :  110
Predicted: 28
SE.PRM.UNER.FE :  110


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Predicted: 28
SE.PRM.UNER :  94
Predicted: 22
SE.PRM.TENR.MA :  110
Predicted: 28


...

Predicted: 15
SE.ADT.1524.LT.FM.ZS :  175
Predicted: 15
SE.ADT.1524.LT.FE.ZS :  175


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Predicted: 15
SP.POP.1564.TO.ZS :  24
Predicted: 0
SP.POP.0014.TO.ZS :  24
Predicted: 0
SL.UEM.TOTL.ZS :  31
Predicted: 0
SL.UEM.TOTL.MA.ZS :  31
Predicted: 0
SL.UEM.TOTL.FE.ZS :  31
Predicted: 0
SL.TLF.TOTL.IN :  31
Predicted: 0
SL.TLF.TOTL.FE.ZS :  31
Predicted: 0
SE.XPD.TOTL.GD.ZS :  98


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Predicted: 19
SE.XPD.TOTL.GB.ZS :  111
Predicted: 16
SE.XPD.TERT.ZS :  112
Predicted: 14
